<h1 style="font-size: 36px; color: #FFD700">1 - QUICKSTART</h1>

Pytorch có 2 nguyên hàm để làm việc với dữ liệu
- torch.utils.data.DataLoader - bọc một đối tượng có thể lặp lại xung quanh Dataset
- torch.utils.data.Dataset - lưu trữ mẫu và nhãn tướng ứng

Dataset là một iterable - đối tượng lặp đi lặp lại, bọc quanh 1 đối tượng có nghĩa là Dataset có thể lặp qua từng mẫu bằng tay. Điều này không hiệu quả nên Dataloader sẽ thực hiện thay tự động và thông minh hơn.

- Tự động chia thành batch
- Shuffle nếu muốn
- Tải song song

In [4]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

- import torch - Tải PyTorch

- from torch import nn - Chứa các công cụ để xây dựng mạng nơ-ron

- from torch.utils.data import DataLoader - Giúp chia dữ liệu thành các batch nhỏ, có thể shuffle và load song song

- from torchvision import datasets -  Import tập các dataset hình ảnh phổ biến

- from torchvision.transforms import ToTensor - Một phép biến đổi ảnh từ định dạng bình thường (PIL/NumPy) sang tensor

Cung cấp các thư viện riêng cho từng miền: TorchText, TorchVision, TorchAudio chứa các dataset. Hướng dẫn này tập trung vào TorchVision

Mỗi TorchVision Datasets bao gồm 2 đối số: transform và target_transform để sửa mẫu và nhãn tương ứng.

In [6]:
# Tải dữ liệu huấn luyện từ datasets
training_data = datasets.FashionMNIST(
    root="data", # thư mục chứa dữ liệu tải về
    train=True, # train = true là ta tải tập train và tập test = False
    download=True, # tải từ internet nếu chưa có dataset
    transform=ToTensor() # chuyển ảnh thành tensor pytorch
)

# Tải tập test
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%
100.0%
100.0%
100.0%


Ảnh gốc trong dataset là định dạng PIL Image. 

ToTensor() trong trường hợp này:
- Chuyển ảnh thành Tensor Pytorch
- Scale pixel từ [0, 255] -> [0.0, 1.0]

batch_size = 64 nghĩa mỗi phần tử iterable dataloader sẽ trả về 1 batch gồm 64 feature và labels - ở đây là 64 tấm ảnh và nhãn

In [8]:
# Truyền Dataset vào Dataloader
batch_size = 64 # số mẫu trên mỗi batch

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size) 
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader: # mỗi vòng lặp trả ra X; 64 ảnh, y 64 nhãn
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Dataloader:
- Tự động chia thành từng bacth gồm 64 tấm ảnh
- trả về (ảnh, nhãn) ở mỗi lần lặp
- có thể shuffle (shuffle=True)

X.shape:
- N: batch size - 64
- C: kênh ảnh - 1 kênh đen/trắng
- H: chiều cao ảnh - 28
- W - chiều rộng ảnh - 28 


y.shape: trả về số nhãn là kiểu số nguyên type torch.int64

Để định nghĩa mạng neural ta sẽ tạo lớp kế thừa từ nn.Module. Để tăng tốc, ta di chuyển nó đến accelerator - bộ tăng tốc. Nếu nó không khả dụng thì dùng CPU.
- Tạo lớp kế thừa từ nn.Module, đn các lớp của mạng trong hàm __init__
- Chỉ định cách dữ liệu đi qua trong hàm forward.

In [10]:
# ktra thiết bị
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# định nghĩa models
class NeuralNetwork(nn.Module):
    def __init__(self): # Khởi tạo
        super().__init__()
        self.flatten = nn.Flatten() # Chuyển từ ảnh 2D -> vector 1D
        self.linear_relu_stack = nn.Sequential( # xếp chồng các lớp liên tiếp
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x): # đn các dữ liệu đi qua mạng
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) # chạy qua từng lớp đã khai báo
        return logits # đầu ra chưa chuẩn hóa -> dùng tính loss (vector 10 chiều)
    
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


nn.Sequential() để xếp chồng các lớp liên tiếp.
- Linear: lớp học có trọng số (fully connected layer)
- ReLU(): hàm kích hoạt để tạo phi tuyến tính, ReLu sẽ tính giá trị đầu ra của mỗi node trước khi truyền sang lớp tiếp theo
- output là 10 lớp (phân loại 10 loại quần áo)

Cách biến đổi từng vector ở từng lớp ví dụ 784 -> 512: Ta sẽ dùng phép nhân ma trận: output = input @ W.T + b
- input: [batch_size, 784]
- W: trọng số, kích thước [512, 784]
- b: vector bias [512].

Ta sẽ dùng 784 input với một bộ weight cho từng node tính từng node như vậy đến khi hết 512 node là ta sẽ có được một lớp vector 512 node.

Để tối ưu hóa thì ta cần một hàm mất mát và một trình tối ưu hóa.

In [11]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

CrossEntropyLoss được dùng cho phân loại nhiều lớp
- Đầu ra là logits chưa qua softmax 
- Nhãn đầu ra là chỉ số class(0, 1, 2, ..., 9)

- torch.optim.SGD(...)
SGD = Stochastic Gradient Descent

Là phương pháp cập nhật trọng số mô hình dựa trên đạo hàm (gradient) của loss

- model.parameters()
Truyền vào tất cả các tham số học được của mô hình (weights và biases)

Optimizer sẽ tự động biết phải cập nhật những gì

- lr=1e-3
Learning rate = 0.001

Tốc độ cập nhật trọng số

Nhỏ quá → học chậm

Lớn quá → học không ổn định

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # chuyển sang chế độ train
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # đưa dữ liệu vào thiết bị
        
        pred = model(X) # truyền X vào mô hình để dự đoán
        loss = loss_fn(pred, y) # trả ra giá trị loss
        
        loss.backward() # tính grad, bước lan truyền ngược trong BP
        optimizer.step() # dựa vào grad vừa tính và cập nhật vào models -. giảm loss
        optimizer.zero_grad() # xóa -> grad cũ nếu không sẽ bị cộng dồn qua mỗi batch
        
        if batch % 100 == 0: # in loss sau mỗi 100 batch
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

size dùng để in tiến độ huấn luyện 

model.train(): phải bật vì Dropout, BatchNorm sẽ hđ khác khi train/test

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # chuyển mô hình sang chế độ đánh giá
    test_loss, correct = 0, 0
    with torch.no_grad(): # tắt grad
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # đưa vào device để tăng hiệu suất
            pred = model(X) # logits đầu ra từ models
            test_loss += loss_fn(pred, y).item() # tính loss của từng bathc rồi cộng dồn
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # Tính số mẫu dự đoán đúng
    
    # Tính AVG loss & accuracy
    test_loss /= num_batches 
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

model.eval(): bắt buộc khi đánh giá mô hình, tắt các chức năng chỉ dùng huấn luyện như Dropout, BatchNorm, ...

with torch.no_grad(): không tính grad
- không cần BP khi test
- tiết kiệm được bộ nhớ và tăng tốc độ 

-> chỉ cần tính loss để so sánh chứ không cần cập nhật lại mô hình 

In [17]:
epochs = 20 # train theo epochs 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!") 

Epoch 1
-------------------------------
loss: 0.631856 [   64/60000]
loss: 0.732025 [ 6464/60000]
loss: 0.505270 [12864/60000]
loss: 0.749367 [19264/60000]
loss: 0.668971 [25664/60000]
loss: 0.638996 [32064/60000]
loss: 0.725437 [38464/60000]
loss: 0.716660 [44864/60000]
loss: 0.704364 [51264/60000]
loss: 0.673693 [57664/60000]
Test Error: 
 Accuracy: 77.0%, Avg loss: 0.662718 

Epoch 2
-------------------------------
loss: 0.610344 [   64/60000]
loss: 0.712576 [ 6464/60000]
loss: 0.489044 [12864/60000]
loss: 0.736095 [19264/60000]
loss: 0.657228 [25664/60000]
loss: 0.627645 [32064/60000]
loss: 0.707528 [38464/60000]
loss: 0.706812 [44864/60000]
loss: 0.693376 [51264/60000]
loss: 0.659354 [57664/60000]
Test Error: 
 Accuracy: 77.6%, Avg loss: 0.648738 

Epoch 3
-------------------------------
loss: 0.590750 [   64/60000]
loss: 0.694575 [ 6464/60000]
loss: 0.474397 [12864/60000]
loss: 0.723791 [19264/60000]
loss: 0.646676 [25664/60000]
loss: 0.617635 [32064/60000]
loss: 0.690840 [38464/

KeyboardInterrupt: 

Vì 1 epoch chưa đủ để mô hình học tốt: Mỗi lần đi qua toàn bộ dữ liệu (1 epoch), mô hình chỉ học được một phần mối quan hệ giữa dữ liệu và nhãn.

Cần lặp đi lặp lại nhiều lần (epoch) để:

- Tối ưu tốt hơn

- Trọng số điều chỉnh dần dần

- Mô hình học được xu hướng sâu hơn

SGD là một thuật toán cập nhật dần từng bước dựa trên mini-batch, nên cần nhiều epoch để hội tụ.

Cách hoạt động:
Mỗi epoch, bạn theo dõi loss / accuracy trên test set

Nếu thấy:

- Accuracy không tăng nữa

- Hoặc loss không giảm nữa

- Hoặc bắt đầu overfitting (loss tăng trên test set)
→ Dừng lại sớm để tiết kiệm thời gian và tránh overfitting

In [18]:
# Lưu models đã train xong Accuracy: 81.5%, AVG loss: 0.525682
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
# Load models từ file weights
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

Test

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
